
Projet de Combinatoire et Énumération
====== 

[Sujet du projet](https://github.com/hivert/CombiFIIL/raw/master/SujetProjet/projet.pdf)

L'objectif de ce projet est d'implémenter un comptage d'objets combinatoire décrits par une grammaire quelconque, on utilisera pour ce faire les méthodes générales de produit cartésien et d'union disjointe vues en cours.



## Questions de cours

### Question 1

Pour les grammaires des arbres et des mots de Fibonnacci, donner dans un tableau pour
n = 0, . . . , 10 les réponses attendue pour la méthode count pour les 8 non terminaux
des mots de Fibonnacci et les 3 non terminaux des arbres binaires

_______________

|         |  0|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10|
|:---     |---|---|---|---|---|---|---|---|---|---|---|
| Fib     |   |   |   |   |   |   |   |   |   |   |   |
| Cas1    |   |   |   |   |   |   |   |   |   |   |   |
| Cas2    |   |   |   |   |   |   |   |   |   |   |   |
| CasAu   |   |   |   |   |   |   |   |   |   |   |   |
| CasBAu  |   |   |   |   |   |   |   |   |   |   |   |

### Question 2

Donner la grammaire de tous les mots sur l’alphabet A,B.

____________


### Question 3

Donner la grammaire des mots de Dyck, c’est-à-dire les mots sur l’alphabet {(, )} et
qui sont correctement parenthésés.

____________


### Question 4

Donner la grammaire de mots sur l’alphabet A,B qui n’ont pas trois lettres consécuti-
vement égales.

____________

### Question 5

Donner la grammaire des palindromes sur l’alphabet A, B, même question sur l’alphabet
A,B,C.

_____________


### Question 6

Donner la grammaire des mots sur l’alphabet A,B qui contiennent autant de lettres A
que de lettres B.

______________


### Question 7

Écrire une fonction qui vérifie qu’une grammaire est correcte, c’est-à-dire que chaque
non-terminal apparaissant dans une règle est bien défini par la grammaire.

____________

## Implémentation des classes de grammaire

Nous commençons par implémenter les différentes classes qui modéliseront la grammaire en Python, on suivra la hierarchie suivante:

![](./combi_class_hierarchy.png)


- __AbstractRule__: Représente une abstraction des règles de la grammaire, en particulier elle implémentera un dictionnaire qui fera référence à la grammaire toute entière ainsi qu'une fonction __set_grammar__ qui permettra de modifier la grammaire (?)
- __ConstructorRule__: Règles de constructions qui répresentent les symboles non-terminaux et qui sont dans notre cas __UnionRule__ et __ProductRule__
    - ProductRule : Cette règle représente le produit cartésien de deux règles, elle hérite de la classe ConstructorRule
    - UnionRule : Il s'agit de l'union disjointe de deux règles, elle hérite également de la classe ConstructorRule
- __ConstantRule__: Règles constantes représentant les symboles terminaux
    - EpsilonRule: Règle représentant le symbole $\epsilon$
    - SingletonRule: Règle représentant un symbole terminal quelconque

In [1]:
from abc import ABCMeta, abstractmethod


class AbstractRule(metaclass=ABCMeta):

    def __init__(self,constant):
        self._grammar = {}
        self._constant = constant

    def _set_grammar(self, gram):
        self._grammar = gram
        
    def isConstant():
        return self._constant


class ConstantRule(AbstractRule):
    def __init__(self, pyobj, valuation):
        super().__init__(True)
        self._object = pyobj
        self.__valuation = valuation
        
    def valuation(self):
        return self.__valuation


class ConstructorRule(AbstractRule):
    def __init__(self, value):
        super().__init__(False)
        self._valuation = float('inf')
        self._parameters = value

    def valuation(self):
        return self._valuation

    @abstractmethod
    def _calc_valuation(self):
        """update valuation"""
        return

    def _update_valuation(self):
        self._grammar[self._parameters[0]]._update_valuation()


## Implémentation des classes héritières et algorithmique

Dans cette partie on implémente les classes Rule qui héritent de nos classes abstraites précedentes, c'est également là qu'on implémentera l'ensemble des fonctions (calcul de valuation, rank, unrank, count, ...)

In [2]:
class UnionRule(ConstructorRule):
    def __init__(self, rule1, rule2):
        super().__init__((rule1, rule2))
        
    def _calc_valuation(self):
        rule1, rule2 = tuple(map(lambda p : self._grammar[p], self._parameters))
        valuation = min(rule1.valuation(), rule2.valuation())
        self._valuation = valuation
        
class ProductRule(ConstructorRule):
    def __init__(self, rule1, rule2, cons):
        super().__init__((rule1, rule2))
        self.__cons = cons
        
    def _calc_valuation(self):
        rule1, rule2 = tuple(map(lambda p : self._grammar[p], self._parameters))
        valuation = rule1.valuation() + rule2.valuation()
        self._valuation = valuation
        
    def cons():
        return self.__cons
    
class EpsilonRule(ConstantRule):
    def __init__(self,obj):
        super().__init__(obj,0)

class SingletonRule(ConstantRule):
    def __init__(self,obj):
        super().__init__(obj,1)

In [3]:
class BinaryTree():
    
    def __init__(self, children = None):
        """
        A binary tree is either a leaf or a node with two subtrees.
        
        INPUT:
            
            - children, either None (for a leaf), or a list of size excatly 2 
            of either two binary trees or 2 objects that can be made into binary trees
        """
        self._isleaf = (children is None)
        if not self._isleaf:
            if len(children) != 2:
                raise ValueError("A binary tree needs exactly two children")
            self._children = tuple(c if isinstance(c,BinaryTree) else BinaryTree(c) for c in children)
        self._size = None
        
    def __repr__(self):
        if self.is_leaf():
            return "leaf"
        return str(self._children)
    
    def __eq__(self, other):
        """
        Return true if other represents the same binary tree as self
        """
        if not isinstance(other, BinaryTree):
            return False
        if self.is_leaf():
            return other.is_leaf()
        return self.left() == other.left() and self.right() == other.right()
    
    
    def left(self):
        """
        Return the left subtree of self
        """
        return self._children[0]
    
    def right(self):
        """
        Return the right subtree of self
        """
        return self._children[1]
    
    def is_leaf(self):
        """
        Return true is self is a leaf
        """
        return self._isleaf
    
    
leaf = BinaryTree()

In [22]:
treeGram = {"Tree" : UnionRule("Node", "Leaf"),
"Node" : ProductRule("Tree", "Tree", lambda c : BinaryTree([c[0],c[1]])),
"Leaf" : SingletonRule(leaf)}

def init_grammar(gram):
    
    for ruleName, ruleDef in gram.items():
        
        if not ruleDef._constant:
            for pr in ruleDef._parameters :
                if pr not in gram:
                    raise ValueError("A rule of the grammar ("+pr+") does not exist")
                
        ruleDef._set_grammar(gram)
        
    init_valuation = [rule.valuation() for rule in gram.values()]

    print(init_valuation)
    
                
        
        

In [23]:
init_grammar(treeGram)

[inf, inf, 1]


In [24]:
for k, v in treeGram.items():
    if not v._constant:
        v._calc_valuation()

In [25]:
[rule.valuation() for rule in treeGram.values()]

[1, 2, 1]

In [26]:
incorrectGram = {"Incorrect" : UnionRule("Incorrect","Incorrect")}

In [27]:
init_grammar(incorrectGram)

[inf]


In [30]:
for k, v in incorrectGram.items():
    if not v._constant:
        v._calc_valuation()

In [31]:
[rule.valuation() for rule in incorrectGram.values()]

[inf]